In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [4]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import ContactAveraging
from june.infection import Infection
from june.infection.symptoms import SymptomsConstant
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import leisure, Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.seed import Seed
from june.policy import Policy, Policies
from june import paths
from june.logger.read_logger import ReadLogger
from june.infection.infection import InfectionSelector
from june.world import generate_world_from_hdf5, generate_world_from_geography

No --data argument given - defaulting to:
/home/arnau/code/JUNE/data
No --configs argument given - defaulting to:
/home/arnau/code/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [26]:
%%time
#geography = Geography.from_file({
#                                "super_area": 
#                                          ["E02003282",
#                                          "E00088544", 
#                                          "E02002560", 
#                                          "E02002559"]
#                                }
#                                )
geography = Geography.from_file(
        {
            "super_area": [
                "E02003282",
                "E02001720",
                "E00088544",
                "E02002560",
                "E02002559",
                "E02004314",
            ]
        }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True, include_commute=True)


2020-06-17 15:52:22,790 - june.demography.geography - INFO - There are 129 areas and 5 super_areas in the world.
2020-06-17 15:52:22,802 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-06-17 15:52:22,871 - june.groups.school - INFO - There are 19 schools in this geography.
2020-06-17 15:52:22,894 - june.groups.school - INFO - No school for the age 0 in this world.
2020-06-17 15:52:22,897 - june.groups.school - INFO - No school for the age 1 in this world.
2020-06-17 15:52:22,900 - june.groups.school - INFO - No school for the age 2 in this world.
2020-06-17 15:52:23,140 - june.groups.carehome - INFO - There are 9 care_homes in this geography.
2020-06-17 15:52:32,608 - june.distributors.worker_distributor - INFO - There are 0 who had to be told to stay real


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



distributing people
[]
[]
[]
[]
[]
[]
[]
[Person(id=47461, sex='f', age=29, ethnicity='A', socioecon_index=8, area=<june.demography.geography.Area object at 0x7f654fcf96a0>, work_super_area=<june.demography.geography.SuperArea object at 0x7f654fce40a0>, sector='P', sub_sector=None, home_city=None, mode_of_transport=<ModeOfTransport Train>, busy=True, subgroups=Activities(residence=<june.groups.group.subgroup.Subgroup object at 0x7f65506a7190>, primary_activity=None, hospital=None, commute=None, rail_travel=None, leisure=None, box=None), health_information=None, susceptibility=1.0, dead=False), Person(id=47487, sex='m', age=53, ethnicity='E', socioecon_index=8, area=<june.demography.geography.Area object at 0x7f654fcf96a0>, work_super_area=<june.demography.geography.SuperArea object at 0x7f654fce40a0>, sector='I', sub_sector=None, home_city=None, mode_of_transport=<ModeOfTransport Bus, minibus or coach>, busy=True, subgroups=Activities(residence=<june.groups.group.subgroup.Subgroup obje

### If it took a long time to run the previous command, it might be a good idea to save the world to reuse it later.

In [27]:
world.to_hdf5("test.hdf5")

If we would like to load the world we saved, we just do

In [28]:
world = generate_world_from_hdf5("test.hdf5")

Loaded chunk 0 of 1
Loaded chunk 0 of 1


100%|██████████| 46784/46784 [00:00<00:00, 112457.25it/s]


you have now a beautiful pre-pandemic world. 

## Commute, travel and leisure

We can add cinemas, pubs, groceries, etc. as well as commute in major cities with

In [29]:
# leisure
world.cinemas = Cinemas.for_geography(geography)
world.pubs = Pubs.for_geography(geography)
world.groceries = Groceries.for_super_areas(world.super_areas,
                                            venues_per_capita=1/500)

In [30]:
# commute
#world.initialise_commuting()

We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [32]:
world.cemeteries = Cemeteries()

# Adding the infection

Now, you can play around with different models of infections. The building blocks for an infection are: Transmission and Symptoms. For now, only SymptomsConstant and TransmissionConstant are in a state to be used.

In [33]:
selector = InfectionSelector.from_file()

In [34]:
selector

# Adding the interaction

In [35]:
interaction = ContactAveraging.from_file(selector=selector)
#interaction = DefaultInteraction.from_file(selector=selector)

Beta are the intensities of the interaction taking place at the different groups

In [36]:
interaction.beta

{'box': 1,
 'pub': 1,
 'grocery': 1,
 'cinema': 1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 1.0,
 'school': 1.0,
 'household': 1.0,
 'university': 1.0}

to modify these, simply do

In [37]:
interaction.beta['household'] *= 2

In [38]:
interaction.beta

{'box': 1,
 'pub': 1,
 'grocery': 1,
 'cinema': 1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 1.0,
 'school': 1.0,
 'household': 2.0,
 'university': 1.0}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [39]:
interaction.alpha_physical

2.0

In [40]:
interaction.alpha_physical /= 2

In [41]:
interaction.alpha_physical

1.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [42]:
seed = Seed(world.super_areas, selector,)

In [43]:
n_cases = 50
seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [44]:
social_distance = Policy(policy="social_distance",
                         start_time=datetime(2020, 3, 25), 
                         end_time=datetime(2020, 4, 1))

In [45]:
policies = Policies.from_file([social_distance])

In [46]:
policies.config['social distancing']

{'alpha factor': 1.0,
 'beta factor': {'box': 0.5,
  'pub': 0.5,
  'grocery': 0.5,
  'cinema': 0.5,
  'commute_unit': 0.5,
  'commute_city_unit': 0.5,
  'hospital': 0.5,
  'care_home': 0.5,
  'company': 0.5,
  'school': 0.5,
  'household': 1.0,
  'university': 0.5}}

# Run the simulation

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [47]:
CONFIG_PATH = "../configs/config_example.yaml"

leisure_instance = leisure.generate_leisure_for_config(
world=world, config_filename = CONFIG_PATH
)
simulator = Simulator.from_file(
     world, interaction, selector, 
    config_filename = CONFIG_PATH,
    leisure = leisure_instance,
    policies = policies
)

In [48]:
%%time
simulator.run()

2020-06-17 15:53:11,495 - june.simulator - INFO - Starting group_dynamics for 70 days at day 0
2020-06-17 15:53:11,498 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 70 days
2020-06-17 15:53:15,233 - june.simulator - INFO - Date = 2020-03-07 03:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:17,957 - june.simulator - INFO - Date = 2020-03-07 06:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:18,378 - june.simulator - INFO - Date = 2020-03-07 09:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:20,875 - june.simulator - INFO - Date = 2020-03-08 00:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:23,200 - june.simulator - INFO - Date = 2020-03-08 03:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:25,530 - june.simulator - INFO - Date = 2020-03-08 06:00:00, number of deaths =  0, number of infected = 50
2020-06-17 15:53:25,953 - june.simulator - INFO - Da

AttributeError: 'numpy.int32' object has no attribute 'subgroups'

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [ ]:
import pandas as pd

In [ ]:
read = ReadLogger()

## Hospital data and how it changed over time

In [ ]:
hospitals_df = read.load_hospital_capacity()

In [ ]:
hospitals_df.head(3)

In [ ]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [ ]:
hospitals_characteristics_df

## where did infections happen?

In [ ]:
loc_df = read.get_locations_infections()

In [ ]:
import matplotlib.ticker as mtick

ax = loc_df['percentage_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')

## rate of infection

In [ ]:
r_df = read.get_r()

In [ ]:
r_df.plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
world_df = read.world_summary()

In [ ]:
world_df.plot()
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.legend()

In [ ]:
area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == "E02002512"].plot()
plt.legend()

## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['infected'].plot(label=name)
plt.axvspan(policies.social_distancing_start, 
            policies.social_distancing_end,
            facecolor='y', alpha=0.2)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['dead'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['susceptible'].plot(label=name)
plt.legend()

## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=200,
                                        n_people=5)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')